# Notebook 5
## This notebook geocodes the addresses for buildings 

* For each year datasheet look for buildings that haven't yet got a geocode run on then and use the lookup on the address.

* Building ID should be either:
    * `philadelphia_building_id` - 2013, 2014
    
  or
  
    * `opa_account_num` - 2015, 2016, 2017

In [1]:
import pandas as pd
import os
import re

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

* get a list of csv files in the `data` folder
    * only want CSV files from the data folder that end in _201*.csv, e.g.
      ```
        energy_usage_large_commercial_buildings_reported_2013.csv
        properties_reported_2015.csv
        ..etc..
      ```

In [2]:
data_files = [f for f in os.listdir('../data') if re.search('_201[3-7].csv',f)]
data_files

['energy_usage_large_commercial_buildings_reported_2014.csv',
 'energy_usage_large_commercial_buildings_reported_2013.csv',
 'properties_reported_2015.csv',
 'properties_reported_2017.csv',
 'properties_reported_2016.csv']

* check the data sheets to see which building id column is used

In [3]:
for dfile in data_files:
    df = pd.read_csv('../data/{}'.format(dfile))
    id_col = 'opa_account_num' if 'opa_account_num' in df.columns else 'philadelphia_building_id'
    print(dfile)
    print('\tBuilding ID is', id_col)
    print('\t', df.columns.sort_values())
    print()

energy_usage_large_commercial_buildings_reported_2014.csv
	Building ID is philadelphia_building_id
	 Index(['district_steam_use_kbtu', 'electricity_use_kbtu', 'energy_star_score',
       'fuel_oil_2_use_kbtu', 'location_1_address', 'natural_gas_use_kbtu',
       'notes', 'number_of_buildings', 'philadelphia_building_id',
       'portfolio_manager_id', 'postal_code',
       'primary_property_type_epa_calculated',
       'property_floor_area_buildings_and_parking_ft', 'property_name',
       'site_eui_kbtu_ft', 'source_eui_kbtu_ft', 'total_ghg_emissions_mtco2e',
       'water_use_all_water_sources_kgal', 'year_built'],
      dtype='object')

energy_usage_large_commercial_buildings_reported_2013.csv
	Building ID is philadelphia_building_id
	 Index(['district_steam_use_kbtu',
       'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
       'energy_star_score', 'fuel_oil_2_use_kbtu', 'geom_address', 'lat',
       'lng', 'natural_gas_use_kbtu', 'notes', 'number_of_buildings',

### Mapping between column names

* The 2013 and 2014 data has different column names - need to rename those columns which are matches to the 2015-2017 data frame


* Unique to 2013:
 ```
 'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
 'geom_address',
 'lat',
 'lng',
 'site_eui_kbtu_ft2',
 'source_eui_kbtu_ft2',
 'the_geom',
 'the_geom_webmercator'
 ```
 

* Unique to 2014:
  ```
  'location_1_address', 
  'site_eui_kbtu_ft', 
  'source_eui_kbtu_ft'
  ```
  
* Unique to 2013-2014:
  ```
 'district_steam_use_kbtu',
 'fuel_oil_2_use_kbtu',
 'location_1_address',
 'number_of_buildings',
 'philadelphia_building_id',
 'primary_property_type_epa_calculated',
 'property_floor_area_buildings_and_parking_ft',
 'site_eui_kbtu_ft',
 'source_eui_kbtu_ft',
 'water_use_all_water_sources_kgal'
 ```

  
* Unique to 2015-2017:
  ```
 'fuel_oil_o2_use_kbtu',
 'num_of_buildings',
 'objectid',
 'opa_account_num',
 'primary_prop_type_epa_calc',
 'site_eui_kbtuft2',
 'source_eui_kbtuft2',
 'steam_use_kbtu',
 'street_address',
 'total_floor_area_bld_pk_ft2',
 'water_use_all_kgal'
 ```

In [4]:
c2013 = ['district_steam_use_kbtu',
       'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
       'energy_star_score', 'fuel_oil_2_use_kbtu', 'geom_address', 'lat',
       'lng', 'natural_gas_use_kbtu', 'notes', 'number_of_buildings',
       'philadelphia_building_id', 'portfolio_manager_id', 'postal_code',
       'primary_property_type_epa_calculated',
       'property_floor_area_buildings_and_parking_ft', 'property_name',
       'site_eui_kbtu_ft2', 'source_eui_kbtu_ft2', 'the_geom',
       'the_geom_webmercator', 'total_ghg_emissions_mtco2e',
       'water_use_all_water_sources_kgal', 'year_built']


c2014 = ['district_steam_use_kbtu', 'electricity_use_kbtu', 'energy_star_score',
       'fuel_oil_2_use_kbtu', 'location_1_address', 'natural_gas_use_kbtu',
       'notes', 'number_of_buildings', 'philadelphia_building_id',
       'portfolio_manager_id', 'postal_code',
       'primary_property_type_epa_calculated',
       'property_floor_area_buildings_and_parking_ft', 'property_name',
       'site_eui_kbtu_ft', 'source_eui_kbtu_ft', 'total_ghg_emissions_mtco2e',
       'water_use_all_water_sources_kgal', 'year_built']

c2015_2017 = ['electricity_use_kbtu', 'energy_star_score', 'fuel_oil_o2_use_kbtu',
       'natural_gas_use_kbtu', 'notes', 'num_of_buildings', 'objectid',
       'opa_account_num', 'portfolio_manager_id', 'postal_code',
       'primary_prop_type_epa_calc', 'property_name', 'site_eui_kbtuft2',
       'source_eui_kbtuft2', 'steam_use_kbtu', 'street_address',
       'total_floor_area_bld_pk_ft2', 'total_ghg_emissions_mtco2e',
       'water_use_all_kgal', 'year_built']



set(c2015_2017).difference(set(c2013).union(set(c2014)))

set(c2014).difference(set(c2013).union(set(c2015_2017)))

set(c2013).difference(set(c2014).union(set(c2015_2017)))

set(c2014).union(set(c2015_2017)).difference(set(c2015_2017))


{'district_steam_use_kbtu',
 'fuel_oil_2_use_kbtu',
 'location_1_address',
 'number_of_buildings',
 'philadelphia_building_id',
 'primary_property_type_epa_calculated',
 'property_floor_area_buildings_and_parking_ft',
 'site_eui_kbtu_ft',
 'source_eui_kbtu_ft',
 'water_use_all_water_sources_kgal'}

### Find unique building ids and drop all other rows

* There seems to be some duplication in the building ids when a group of buildings, i.e. campus buildings, are considered a unit. The easiest approach is just to drop these rows from the data frame so that the building id is really unique.


* Also some rows have missing building ids or have non numeric values in them, `Many` or `Multiple`. These rows should also be dropped from the data frame 

In [5]:
def clean_up_dataframe(df):

    building_id_col = 'opa_account_num' if 'opa_account_num' in df.columns else 'philadelphia_building_id'
    df.rename(columns={building_id_col: 'building_id'}, inplace=True)
    
    building_ids = df['building_id'].value_counts().reset_index()
    keep_ids = building_ids[building_ids['building_id']==1]['index']
    
    missing_filter = -df['building_id'].isnull()
    keep_filter = df['building_id'].isin(keep_ids)
    multiple_filter = -df['building_id'].isin(['Many','Multiple'])

    df=df[missing_filter & keep_filter & multiple_filter]
    
    try:
        semicolon_filter = (df['building_id'].str.contains(';')) | (df['building_id'].str.contains(','))
        df=df[-semicolon_filter]
    except:
        pass
    
    df['building_id']=df['building_id'].astype(int)
    return df

<div class="alert alert-info">
    <li>the list of csv files included things like `means.csv` that aren't the actual datasheets so needed to adjust so it is just the 2013-2017 datasheets</li>
</div>

In [6]:
dframes=[]
for dfile in data_files:
    year = dfile.split('_')[-1][:-4]
    print(year, dfile)
    df = pd.read_csv('../data/{}'.format(dfile))
    df_clean = clean_up_dataframe(df)
    df_clean['year']=year
    dframes.append(df_clean)

2014 energy_usage_large_commercial_buildings_reported_2014.csv
2013 energy_usage_large_commercial_buildings_reported_2013.csv
2015 properties_reported_2015.csv
2017 properties_reported_2017.csv
2016 properties_reported_2016.csv


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
full_df=pd.concat(dframes)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
full_df.sample(30)

,building_id,district_steam_use_kbtu,electricity_use_grid_purchase_and_generated_from_onsite_renewab,electricity_use_kbtu,energy_star_score,fuel_oil_2_use_kbtu,fuel_oil_o2_use_kbtu,geom_address,lat,lng,...,steam_use_kbtu,street_address,the_geom,the_geom_webmercator,total_floor_area_bld_pk_ft2,total_ghg_emissions_mtco2e,water_use_all_kgal,water_use_all_water_sources_kgal,year,year_built
908,777630000,Not Available,6276.7,NaN,Not Available,Not Available,NaN,1801 North 9th Street,39.926824,-75.160476,...,NaN,NaN,0101000020E6100000D05A9B3D45CA52C058982628A2F6...,0101000020110F00009C4D717BBAEA5FC14B358C8B8485...,NaN,1.3,NaN,0.0,2013,1900
687,771634900,NaN,NaN,33324905.1,94,NaN,NaN,NaN,NaN,NaN,...,NaN,440 N BROAD ST,NaN,NaN,835000.0,4461.1,5055.9,NaN,2015,2005
1291,884209570,NaN,NaN,3120521.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2950 KIRKBRIDE ST,NaN,NaN,256000.0,461.7,373.9,NaN,2016,1963
103,882506315,16933926.5,22021754.3,NaN,95,Not Available,NaN,801 Market Street,39.951193,-75.153667,...,NaN,NaN,0101000020E6100000686473AFD5C952C0888EFEB3C0F9...,0101000020110F0000296A8AFEFCE95FC155C10D17F988...,NaN,4288.1,NaN,9039.9,2013,1909
1417,881091000,NaN,NaN,49460.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8875 RIDGE AVE,NaN,NaN,71040.0,5.5,3.8,NaN,2017,1980
675,884220900,NaN,NaN,398743.4,100,NaN,NaN,NaN,NaN,NaN,...,NaN,6225 OXFORD AVE,NaN,NaN,103528.0,53.4,51.0,NaN,2015,2000
722,881021500,NaN,NaN,1606574.3,46,NaN,NaN,NaN,NaN,NaN,...,NaN,301 S 19TH ST,NaN,NaN,78750.0,454.9,2005.5,NaN,2016,1933
1265,884123105,NaN,NaN,2468801.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3605 GRAYS FERRY AVE,NaN,NaN,80785.0,317.1,587.6,NaN,2017,1984
746,784093000,Not Available,2548471.6,NaN,74,Not Available,NaN,1501 W DIAMOND ST,39.984998,-75.158336,...,NaN,NaN,0101000020E6100000F48EB02E22CA52C00815DF6714FE...,0101000020110F0000FA83A4EE7EEA5FC142BC8D9BC48D...,NaN,478.2,NaN,362.7,2013,1968
214,784484000,NaN,NaN,43570400.6,44,NaN,3631608.1,NaN,NaN,NaN,...,NaN,5000 WISSAHICKON AVE,NaN,NaN,449021.0,5153.7,9108.5,NaN,2017,1998


In [9]:
full_df['building_id'].value_counts().value_counts(sort=False)

1    192
2    560
3    183
4    329
5    542
Name: building_id, dtype: int64

In [10]:
full_df.shape

(5887, 38)

<div class="alert alert-warning">
    <li><code>full_df</code> has all dataframes combined - be still need to rename the columns</li>
    <li>But for now can use it to get unique building ids and street addresses that need then to have geo lookup</li>
</div>

In [11]:
full_df.columns

Index(['building_id', 'district_steam_use_kbtu',
       'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
       'electricity_use_kbtu', 'energy_star_score', 'fuel_oil_2_use_kbtu',
       'fuel_oil_o2_use_kbtu', 'geom_address', 'lat', 'lng',
       'location_1_address', 'natural_gas_use_kbtu', 'notes',
       'num_of_buildings', 'number_of_buildings', 'objectid',
       'portfolio_manager_id', 'postal_code', 'primary_prop_type_epa_calc',
       'primary_property_type_epa_calculated',
       'property_floor_area_buildings_and_parking_ft', 'property_name',
       'site_eui_kbtu_ft', 'site_eui_kbtu_ft2', 'site_eui_kbtuft2',
       'source_eui_kbtu_ft', 'source_eui_kbtu_ft2', 'source_eui_kbtuft2',
       'steam_use_kbtu', 'street_address', 'the_geom', 'the_geom_webmercator',
       'total_floor_area_bld_pk_ft2', 'total_ghg_emissions_mtco2e',
       'water_use_all_kgal', 'water_use_all_water_sources_kgal', 'year',
       'year_built'],
      dtype='object')

In [12]:
building_ids = full_df['building_id'].value_counts().reset_index()
building_ids.head()

,index,building_id
0,882055181,5
1,786154910,5
2,884130000,5
3,883704600,5
4,884267500,5


In [13]:
rows_without_geo=full_df[full_df['lat'].isnull()]

In [14]:
rows_without_geo.shape

(4938, 38)

In [15]:
rows_without_geo['building_id'].nunique()

1757

<div class="alert alert-info">
    <li>So there are 1757 buildings that need to have geolookup</li>
    <li>Need to get building_id and street address from appropriate columns into a new list</li>
    <li>Write this list to another file and then do batches of lookup</li>
    <li>Then can merge all the data back together</li>
</div>

In [16]:
rows_to_geolookup=rows_without_geo.drop_duplicates(subset=['building_id'])

In [17]:
rows_to_geolookup.shape

(1757, 38)

In [18]:
rows_to_geolookup.columns

Index(['building_id', 'district_steam_use_kbtu',
       'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
       'electricity_use_kbtu', 'energy_star_score', 'fuel_oil_2_use_kbtu',
       'fuel_oil_o2_use_kbtu', 'geom_address', 'lat', 'lng',
       'location_1_address', 'natural_gas_use_kbtu', 'notes',
       'num_of_buildings', 'number_of_buildings', 'objectid',
       'portfolio_manager_id', 'postal_code', 'primary_prop_type_epa_calc',
       'primary_property_type_epa_calculated',
       'property_floor_area_buildings_and_parking_ft', 'property_name',
       'site_eui_kbtu_ft', 'site_eui_kbtu_ft2', 'site_eui_kbtuft2',
       'source_eui_kbtu_ft', 'source_eui_kbtu_ft2', 'source_eui_kbtuft2',
       'steam_use_kbtu', 'street_address', 'the_geom', 'the_geom_webmercator',
       'total_floor_area_bld_pk_ft2', 'total_ghg_emissions_mtco2e',
       'water_use_all_kgal', 'water_use_all_water_sources_kgal', 'year',
       'year_built'],
      dtype='object')

In [19]:
[c for c in rows_to_geolookup.columns if c.count('address')>0 or c.count('code')>0]

['geom_address', 'location_1_address', 'postal_code', 'street_address']

In [20]:
to_lookup=rows_to_geolookup[['building_id',
                  'location_1_address', 'street_address', 'postal_code']]

to_lookup.loc[to_lookup['street_address'].isnull(), 'street_address'] = to_lookup[to_lookup['street_address'].isnull()]['location_1_address']

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
to_lookup.drop(columns=['location_1_address'], inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
to_lookup['lat']=None
to_lookup['lng']=None

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
to_lookup.sample(20)

,building_id,street_address,postal_code,lat,lng
757,881822370,4035 PARRISH ST,19104,None,None
900,886662700,6300 OLD YORK RD,19141,None,None
690,884249500,4219 Van Kirk St.,19135,None,None
901,881551130,1901-39 CALLOWHILL ST,19130,None,None
992,888058983,1401 WALNUT ST,19102,None,None
545,883743000,8900 Bartram Avenue,19153,None,None
460,784615800,2700 W DAUPHIN ST,19132,None,None
994,782513700,2600 Ben Franklin Pkwy,19130,None,None
551,771635100,447 N. Broad St.,19123,None,None
858,786482200,3001 RYAN AVE,19152,None,None


In [24]:
to_lookup.to_csv('../data/buildings_needing_geoinfo.csv', index=False)

## Do geolookup for buildings missing lat and long in the datasheets

* Now can use the `to_lookup` data frame which is a unique list of building ids to do geolookup

### Setup `lookup` object

In [25]:
geolocator = Nominatim(user_agent="application")
lookup = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [26]:
def geo_code_selection(df, sample_size=1577):
    
    need_coding = df[(df['lat'].isnull()) & (df['lng'].isnull())]
    
    print('There are {} buildings to code'.format(need_coding.shape[0]))
    
    print('\nSelecting {} to code'.format(sample_size))
    
    to_code = need_coding.sample(sample_size)

    for row in to_code.to_dict(orient='records'):

        building_id=row['building_id']


        street = row['street_address'].title()

        street = re.sub('^(\d+)-\d+','\\1', street)

        zip = str(int(row['postal_code']))

        address = "{}, PA {}".format(street, zip)

        print(address)
        location = lookup(address, language='en', exactly_one=True)

        try:
            lat=location.latitude
            lng=location.longitude
            df.loc[df['building_id']==building_id,'lat']=lat
            df.loc[df['building_id']==building_id,'lng']=lng
        except:
            print("Couldn't find geolocation")
    

In [27]:
geo_code_selection(to_lookup, sample_size=100)
to_lookup.to_csv('../data/buildings_needing_geoinfo.csv', index=False)

There are 1757 buildings to code

Selecting 100 to code
1520 Locust Street, PA 19102


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


7400 Holstein Ave., PA 19153
1201 Washington Ave, PA 19147
725 South 5Th St, PA 19147
5000 Oxford Ave, PA 19124
2100 Kubach Road, PA 19116
1327 Locust St, PA 19107
1666 Callowhill St, PA 19130
6498 Ridge Ave, PA 19128
3615 Market St., PA 19104
2901 S 10Th Street, PA 19148
521 S Broad St, PA 19147
2017 Chestnut St, PA 19103
600 N 34Th St, PA 19104
1751 Callowhill St, PA 19130
1651 Christopher Columbus Blvd, PA 19148
1700 Bigler St, PA 19145
160 E Erie Ave, PA 19134
5400 E Roosevelt Blvd, PA 19124
Couldn't find geolocation
108 Arch St, PA 19106
1821 E Hagert St, PA 19125
20 S 36Th St, PA 19104
2967 W School House Ln, PA 19144
5720 Media St, PA 19131
4901 Rising Sun Ave, PA 19120
833 Chestnut Street, PA 19107
201 Sunset Ave, PA 19118
5675 Rising Sun Ave, PA 19120
626 Dickinson St, PA 19147
1007 W. Lehigh Avenue, PA 19133
301 Race St, PA 19106
4901 Stenton Ave, PA 19144
1508 W Allegheny Ave, PA 19132
7100 Henry Ave, PA 19128
11620 Caroline Rd., PA 19154
5515 Wissahickon Ave, PA 19144
13500

ValueError: invalid literal for int() with base 10: '19126-2233'

In [28]:
to_lookup

,building_id,street_address,postal_code,lat,lng
0,785548000,450 W NEDRO AVE,19120,None,None
1,884240500,10814 NE AVENUE,19116,None,None
2,884114205,"4123 ""G"" St.",19124,None,None
3,784463600,4667 UMBRIA,19143,None,None
4,784378000,6000 RIDGE AVE,19128,None,None
5,783183400,5001 GREENWAY AVE,19143,None,None
6,783087800,6001 CEDAR AVE,19143,None,None
7,883359300,1163-65 S. BROAD STREET,19147,None,None
8,883013200,117-31 N 8th Street,19106,None,None
9,882167505,6701 Essingtion ave,19153,None,None


In [ ]:
astr='1147 N 4Th St, PA 19123'

location = lookup(astr, language='en', exactly_one=True)
location

Location(1147, North 4th Street, Olde Kensington, Philadelphia, Philadelphia County, Pennsylvania, 19123, United States of America, (39.968576244898, -75.143281755102, 0.0))